In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
%matplotlib inline
from analysis.lib.fitting import fit, common

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data\TPQI_combi'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\TPQI_combi'
from analysis.lib.purification import purify_TPQI as TPQI; reload(TPQI)

# Filter by CR checks

In [ ]:
### define parameters
start_ch0 = 2436.5e3#+700e3
start_offset_ch1 = 0.0e3
window_length = 50e3
pulse_sep = 700e3
number_of_pulses = 10

# CR check filtering
cr_check_min_p = 0
cr_check_min_s = 0
filter_before_or_after = 'after'


In [ ]:
### get coincidences
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
reload(TPQI)

dts, filtered_dts, cr_checks = TPQI.TPQI_analysis_w_CR_check_filtering(folder_lt4,start_ch0,
                                                                       start_ch0+start_offset_ch1,
                                                                       window_length,
                                                                       pulse_sep,
                                                                       number_of_pulses,
                                                                       load_cr_check = True,
                                                                       Base_Folder_secondary = folder_lt3, 
                                                                       cr_check_min_p = cr_check_min_p,
                                                                       cr_check_min_s =cr_check_min_s,
                                                                       filter_before_or_after = filter_before_or_after,
                                                                       load_TPQI_attempts = True,
                                                                       min_attempts = 0,
                                                                       max_attempts = 99,
                                                                       Verbose = False)
len(dts),len(filtered_dts)

In [ ]:
### plot result
hist_binsize = 1
bins = np.arange(1,40,hist_binsize)
y,x=np.histogram(cr_checks[:,0])
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(2,2,1)
ax.plot(bin_centres,y,'-o')
ax.set_xlabel('CR before')
ax.set_ylabel('Occurencces')
# ax.set_range
ax.set_title('LT4')

hist_binsize = 1
bins = np.arange(1,40,hist_binsize)
y,x=np.histogram(cr_checks[:,1])
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(2,2,2)
ax.plot(bin_centres,y,'-o')
ax.set_xlabel('CR after')
ax.set_ylabel('Occurencces')
# ax.set_range
ax.set_title('LT4')

hist_binsize = 1
bins = np.arange(1,40,hist_binsize)
y,x=np.histogram(cr_checks[:,2])
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(2,2,3)
ax.plot(bin_centres,y,marker = 'o')
ax.set_xlabel('CR before')
ax.set_ylabel('Occurencces')
# ax.set_range
ax.set_title('LT3')

hist_binsize = 1
bins = np.arange(1,40,hist_binsize)
y,x=np.histogram(cr_checks[:,3])
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(2,2,4)
ax.plot(bin_centres,y,marker = 'o')
ax.set_xlabel('CR after')
ax.set_ylabel('Occurencces')
# ax.set_range
ax.set_title('LT3')

plt.show()
plt.close('all')

In [ ]:
### plot result
hist_binsize = 6
bins = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,hist_binsize)
y,x=np.histogram(filtered_dts, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(bin_centres,y)
ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences in bin')
# ax.set_range
ax.set_title('TPQI')
ax.set_xlim([-50,50])
plt.show()
plt.close('all')

In [ ]:
def fit_tpqi(g_a,n_pulses):
    """
    fitfunction for TPQI
        y(x) = a * (n_pulses - x)/n_pulses

    I.g.:
        g_a : amplitude
    """
    fitfunc_str = 'a'

    a = fit.Parameter(g_a, 'a')
    n_pulses = fit.Parameter(n_pulses, 'n_pulses')
    p0 = [a, n_pulses]

    def fitfunc(x):
        return a() * (n_pulses() - x)/n_pulses()

    return p0, fitfunc, fitfunc_str


counts = np.zeros(number_of_pulses)
diff =  range(1,number_of_pulses+1)
for z in diff:
    c_left = np.sum((filtered_dts<(200-z*pulse_sep/1e3)) & (filtered_dts>(-200-z*pulse_sep/1e3)))
    c_right = np.sum((filtered_dts<(200+z*pulse_sep/1e3)) & (filtered_dts>(-200+z*pulse_sep/1e3)))
    counts[z-1] = (c_left + c_right)/2
    
reload(fit)
fitted = fit.fit1d(diff,counts,fit_tpqi,300,10,ret=True)

plt.plot(diff, counts ,'o')
plt.plot(diff,fitted['fitfunc'](diff))
plt.xlabel('Pulse difference')
plt.ylabel('Coincidences')

fitted_counts_in_next_pulse = fitted['params'][0]
print 'Fitted coincidences with next pulse', fitted_counts_in_next_pulse

In [ ]:
counts_in_next_pulse = (np.sum((filtered_dts<-400) & (filtered_dts>-800))+np.sum((filtered_dts>400) & (filtered_dts<800)))/2
counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))


print 'counts left',np.sum((filtered_dts<-400) & (filtered_dts>-800))
print 'counts right',np.sum((filtered_dts>400) & (filtered_dts<800))
print 'averaged coincidences with following pulses',counts_in_next_pulse 
print 'coincidences in same pulse',counts_in_same_pulse

print fitted_counts_in_next_pulse
tail_lt3 = 9.1975
tail_lt4 = 5.1514

p1 = tail_lt3
p2 = tail_lt4

Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
print 'assumed tail LT3/LT4',tail_lt3,tail_lt4
print 'distiungishable expected vis', Visibility
print 'measured Visibility', counts_in_same_pulse/float(counts_in_next_pulse)
print 'measured Visibility', counts_in_same_pulse/float(fitted_counts_in_next_pulse)
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(fitted_counts_in_next_pulse))/Visibility


# Filter by adwin comm time

In [ ]:
### define parameters
start_ch0 = 2436.5e3#+700e3
start_offset_ch1 = 0.0e3
window_length = 50e3
pulse_sep = 700e3
number_of_pulses = 10

# Adwin comm time filtering
adwin_com_max_p = 1000
adwin_com_max_s = 1000

In [ ]:
### get coincidences
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
reload(TPQI)

dts, filtered_dts,cr_checks, adwin_com_times = TPQI.TPQI_analysis_w_CR_check_filtering(folder_lt4,start_ch0,
                                                                       start_ch0+start_offset_ch1,
                                                                       window_length,
                                                                       pulse_sep,
                                                                       number_of_pulses,
                                                                       load_adwin_comm_time = True,
                                                                       Base_Folder_secondary = folder_lt3, 
                                                                       adwin_com_max_p = adwin_com_max_p,
                                                                       adwin_com_max_s =adwin_com_max_s)
len(dts),len(filtered_dts)

In [ ]:
### plot result
bins = np.arange(0,10,0.1)
y,x=np.histogram(adwin_com_times[:,0],bins = bins)

fig = plt.figure()
ax = fig.add_subplot(2,1,1)
ax.plot(bins[:-1],y,'-o')
ax.set_xlabel('Adwin com time')
ax.set_ylabel('Occurencces')
# ax.set_range
ax.set_title('LT3')

y,x=np.histogram(adwin_com_times[:,1])
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(2,1,2)
ax.plot(bin_centres,y,'-o')
ax.set_xlabel('Adwin com time')
ax.set_ylabel('Occurencces')
# ax.set_range
ax.set_title('LT4')

plt.show()
plt.close('all')

In [ ]:
### plot result
hist_binsize = 6
bins = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,hist_binsize)
y,x=np.histogram(filtered_dts, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(bin_centres,y)
ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences in bin')
# ax.set_range
ax.set_title('TPQI')
ax.set_xlim([-50,50])
plt.show()
plt.close('all')

In [ ]:
def fit_tpqi(g_a,n_pulses):
    """
    fitfunction for TPQI
        y(x) = a * (n_pulses - x)/n_pulses

    I.g.:
        g_a : amplitude
    """
    fitfunc_str = 'a'

    a = fit.Parameter(g_a, 'a')
    n_pulses = fit.Parameter(n_pulses, 'n_pulses')
    p0 = [a, n_pulses]

    def fitfunc(x):
        return a() * (n_pulses() - x)/n_pulses()

    return p0, fitfunc, fitfunc_str


counts = np.zeros(number_of_pulses)
diff =  range(1,number_of_pulses+1)
for z in diff:
    c_left = np.sum((filtered_dts<(200-z*pulse_sep/1e3)) & (filtered_dts>(-200-z*pulse_sep/1e3)))
    c_right = np.sum((filtered_dts<(200+z*pulse_sep/1e3)) & (filtered_dts>(-200+z*pulse_sep/1e3)))
    counts[z-1] = (c_left + c_right)/2
    
reload(fit)
fitted = fit.fit1d(diff,counts,fit_tpqi,300,10,ret=True)

plt.plot(diff, counts ,'o')
plt.plot(diff,fitted['fitfunc'](diff))
plt.xlabel('Pulse difference')
plt.ylabel('Coincidences')

fitted_counts_in_next_pulse = fitted['params'][0]
print 'Fitted coincidences with next pulse', fitted_counts_in_next_pulse

In [ ]:
counts_in_next_pulse = (np.sum((filtered_dts<-400) & (filtered_dts>-800))+np.sum((filtered_dts>400) & (filtered_dts<800)))/2
counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))


print 'counts left',np.sum((filtered_dts<-400) & (filtered_dts>-800))
print 'counts right',np.sum((filtered_dts>400) & (filtered_dts<800))
print 'averaged coincidences with following pulses',counts_in_next_pulse 
print 'coincidences in same pulse',counts_in_same_pulse

print fitted_counts_in_next_pulse
tail_lt3 = 9.1975
tail_lt4 = 5.1514

p1 = tail_lt3
p2 = tail_lt4

Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
print 'assumed tail LT3/LT4',tail_lt3,tail_lt4
print 'distiungishable expected vis', Visibility
print 'measured Visibility', counts_in_same_pulse/float(counts_in_next_pulse)
print 'measured Visibility', counts_in_same_pulse/float(fitted_counts_in_next_pulse)
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(fitted_counts_in_next_pulse))/Visibility
